In [1]:
from typing import *
from operator import *

import pandas as pd

In [2]:
scenarios_to_average_over = [
    "Binary sequences: Few long i.i.d. sequences",
    "Binary sequences: True distribution is not i.i.d. sequence",
    "Random walk: Many short sequences",
    "Random walk: Few long sequences",
    "Random walk with memory: Many short sequences",
    "Random walk with memory: Few long sequences",
    "Random 2nd-order MC: Many short sequences",
    "Random 2nd-order MC: Few long sequences",
    "Random 2nd-order MC: Few short sequences",
    "Random MC w/ varied initial dist: Many short sequences",
    "Random MC w/ varied initial dist: Few long sequences",
    "Random MC w/ varied length dist",
]

In [3]:
suite_results = pd.read_csv("../../figures/suite-output.csv")

In [4]:
name_map = dict()
for k, k_ in [("ExpDist", "Hamming"), ("CSK", "CSK")]:
    name_map |= {
        **{
            "{Stein(k=" + k + ": op=__mathrm{ZS}_" + str(i) + "_)}":
                (k_ + r" $\mathrm{ZS}$", i)
            for i in [1, 3, 5, 7, 9]
        },
        "{Stein(k=" + k + ": op=__mathrm{ZS}_{_infty:b}_)}":
            (k_ + r" $\mathrm{ZS}$", 13),
        **{
            "{Stein(k=" + k + ": op=__mathrm{ZS}_" + str(i) + "'_)}":
                (k_ + r" $\mathrm{ZS}'$", i)
            for i in [1, 3, 5, 7, 9]
        },
        "{Stein(k=" + k + ": op=__mathrm{ZS}_{_infty:b}'_)}":
            (k_ + r" $\mathrm{ZS}'$", 13),
        **{
            "{Stein(k=" + k + ": op=__mathrm{ZS}_" + str(i) + "''_)}":
                (k_ + r" $\mathrm{ZS}''$", i)
            for i in [1, 3, 5, 7, 9]
        },
        "{MMD(k=" + k + ": n=100)}": (k_ + " MMD", 14),
    }

In [5]:
set(name_map.keys()) - set(suite_results["Test"].unique().tolist())

set()

In [6]:
def _tmp():
    df = suite_results[lambda x: x["Scenario"].isin(scenarios_to_average_over)]
    df = df.groupby("Test")["rejection_rate"].mean().loc[name_map.keys()]
    df.index = pd.MultiIndex.from_tuples(df.index.map(name_map).tolist())
    df = df.rename_axis(["Test", "J"]).unstack("Test")
    display(df)


_tmp()

Test  CSK $\mathrm{ZS}$  CSK $\mathrm{ZS}'$  CSK $\mathrm{ZS}''$  CSK MMD  \
J                                                                           
1              0.172292            0.216458             0.076667      NaN   
3              0.299583            0.326042             0.231875      NaN   
5              0.438750            0.439167             0.347708      NaN   
7              0.491667            0.496250             0.418333      NaN   
9              0.523958            0.535000             0.479167      NaN   
13             0.624792            0.621250                  NaN      NaN   
14                  NaN                 NaN                  NaN    0.575   

Test  Hamming $\mathrm{ZS}$  Hamming $\mathrm{ZS}'$  Hamming $\mathrm{ZS}''$  \
J                                                                              
1                  0.062292                0.083542                 0.054167   
3                  0.102292                0.127917                 0.100208   
5                  0.121667                0.139167                 0.117917   
7                  0.136667                0.162083                 0.126875   
9                  0.131458                0.172708                 0.139375   
13                 0.159583                0.176667                      NaN   
14                      NaN                     NaN                      NaN   

Test  Hamming MMD  
J                  
1             NaN  
3             NaN  
5             NaN  
7             NaN  
9             NaN  
13            NaN  
14          0.085

In [9]:
def _tmp():
    df = suite_results[lambda x: x["Scenario"].isin(scenarios_to_average_over) & x["Test"].isin(name_map.keys())]
    df = df.set_index(["Test", "Scenario"])["rejection_rate"].unstack("Scenario")
    df.index = pd.MultiIndex.from_tuples(df.index.map(name_map).tolist())
    df = df.rename_axis(["Test", "J"]).sort_index()
    int_to_latex_string = lambda x: x if x < 12 else {13: r"$\infty$", 14: ""}[x]
    new_level = df.index.get_level_values("J").map(int_to_latex_string)
    df.index = pd.MultiIndex.from_arrays([df.index.get_level_values("Test"), new_level])
    df = df.T
    col_groups = [
        [c for c in df.columns if "CSK" in c[0]],
        [c for c in df.columns if "Hamming" in c[0]],
    ]
    with open("experiments-appendix-scenarios-table.tex", "w") as f:
        with pd.option_context("styler.format.escape", None):
            for group in col_groups:
                print(df[group].style.format(precision=3).to_latex(), file=f)
    display(df)


_tmp()

Test                                               CSK $\mathrm{ZS}$          \
J                                                                  1       3   
Scenario                                                                       
Binary sequences: Few long i.i.d. sequences                   0.2375  0.4150   
Binary sequences: True distribution is not i.i....            0.0750  0.0225   
Random 2nd-order MC: Few long sequences                       0.0400  0.1575   
Random 2nd-order MC: Few short sequences                      0.0650  0.1175   
Random 2nd-order MC: Many short sequences                     0.1775  0.4200   
Random MC w/ varied initial dist: Few long sequ...            0.0525  0.0650   
Random MC w/ varied initial dist: Many short se...            0.0750  0.0525   
Random MC w/ varied length dist                               0.0325  0.0175   
Random walk with memory: Few long sequences                   0.2200  0.3225   
Random walk with memory: Many short sequences                 0.5250  0.7100   
Random walk: Few long sequences                               0.1300  0.3825   
Random walk: Many short sequences                             0.4375  0.9125   

Test                                                                        \
J                                                        5       7       9   
Scenario                                                                     
Binary sequences: Few long i.i.d. sequences         0.7150  0.8700  0.9325   
Binary sequences: True distribution is not i.i....  0.0500  0.0300  0.0300   
Random 2nd-order MC: Few long sequences             0.2825  0.4375  0.5000   
Random 2nd-order MC: Few short sequences            0.2250  0.2800  0.3575   
Random 2nd-order MC: Many short sequences           0.7200  0.8350  0.9075   
Random MC w/ varied initial dist: Few long sequ...  0.0600  0.0675  0.0475   
Random MC w/ varied initial dist: Many short se...  0.1025  0.0475  0.0600   
Random MC w/ varied length dist                     0.0125  0.0175  0.0250   
Random walk with memory: Few long sequences         0.4875  0.5700  0.5800   
Random walk with memory: Many short sequences       0.8925  0.9425  0.9675   
Random walk: Few long sequences                     0.7400  0.8100  0.8800   
Random walk: Many short sequences                   0.9775  0.9925  1.0000   

Test                                                         \
J                                                  $\infty$   
Scenario                                                      
Binary sequences: Few long i.i.d. sequences          1.0000   
Binary sequences: True distribution is not i.i....   0.0400   
Random 2nd-order MC: Few long sequences              0.9075   
Random 2nd-order MC: Few short sequences             0.4775   
Random 2nd-order MC: Many short sequences            0.9725   
Random MC w/ varied initial dist: Few long sequ...   0.0425   
Random MC w/ varied initial dist: Many short se...   0.0500   
Random MC w/ varied length dist                      0.0350   
Random walk with memory: Few long sequences          0.9775   
Random walk with memory: Many short sequences        0.9950   
Random walk: Few long sequences                      1.0000   
Random walk: Many short sequences                    1.0000   

Test                                               CSK $\mathrm{ZS}'$          \
J                                                                   1       3   
Scenario                                                                        
Binary sequences: Few long i.i.d. sequences                    0.2425  0.5075   
Binary sequences: True distribution is not i.i....             0.0325  0.0075   
Random 2nd-order MC: Few long sequences                        0.0750  0.1725   
Random 2nd-order MC: Few short sequences                       0.0675  0.1550   
Random 2nd-order MC: Many short sequences                      0.1750  0.4900   
Random MC w/ varied initial dist: 